# Final Project for Adaptive Decision Making
### Modeling Behavioral Data from Attention Decay Study with Single-boundary DDM
Group Members: Oscar, Emilio, and William

# Background

# Problem 

+ **Research Question 1 ** What can DDM Modeling tell us about the cognitive process of perceptual decision making?
+ **Research Question 2 ** Can we fit a single-boundary DDM to our empirical Reaction Time (RT) distribution?
+ **Research Question 3 ** Can we Modify the DDM to account for demonstrated attention decay in our empirical data? Will this produce a better fit? 

# Modeling

#### Single decision boundary DDM
This was created by basically ignoring the lower or "no" decision boundary. The modifications to the general DDM algorithm are such:
- If the evidence ever falls below zStart, set the evidence to zStart
- Remove the loop breaking logic for a "no" decision

#### Decay parameter and decay acceleration
We wanted to build in the parameters for attention decay. The intuition is that people will lose focus and pay less attention. The amount of attention that they give to the task will lessen over time such that the speed of evidence gathering is decreasing as the number of trials increases. 
We decided to go with a simple decay model:
- Each trial has an associated decay such that at each step of the evidence gathering, the decision to save a drowning face will get dragged towards "no decision made" by the decay
- After each trial, the amount of decay increases by the decay acceleration

#### Simulating the DDM
Once these models were made, we created algorithms for calculating the chi-square formulas for the dataframe that we had from the behavioral study and the dataframe that we generated with these DDM simulations.

From there we can simulate the DDM and compare it against the behavioral dataframes and tune the parameters to minimize the chi-square and maximize the p-value.

### Do all of our main imports first

In [ ]:
%cd ..

In [ ]:
from __future__ import division
from ADMCode import visualize as vis

import numpy as np
import pandas as pd
import numba as nb

from numba.decorators import jit
from numpy.random import random_sample
from numba import float64, int64, vectorize, boolean

from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter('ignore', np.RankWarning)
warnings.filterwarnings("ignore", module="matplotlib")
warnings.filterwarnings("ignore")
sns.set(style='white', font_scale=1.3)

%matplotlib inline

Trial = 0

### Create a single decision barrier DDM

In [ ]:
@jit(nb.typeof((1.0, 1.0))(float64[:], float64[:], float64[:]), nopython=True)
def sim_ddm(parameters, rProb, trace):

    # extract parameters
    a, tr, v, z, si, dx, dt = parameters

    # convert drift-rate into a probability,
    # & scale by sigma (si) and timestep (dt)
    # if v > 0, then 0.5 < vProb < 1.0
    # if v < 0, then 0.0 < vProb < 0.5
    vProb = .5 * (1 + (v * np.sqrt(dt))/si)

    # define starting point with respect to boundary height
    zStart = z * a

    #initialize evidence variable at zStart
    evidence = zStart
    trace[0] = evidence

    # define deadline (max time allowed for accumulation)
    deadline = trace.size

    for nsteps in range(1, deadline):
        # sample a random probability (r) and compare w/ vProb
        if rProb[nsteps] < vProb:
            # if r < vProb, step evidence up (towards a)
            evidence += dx
        else:
            # if r > vProb, step evidence down (towards 0)
            evidence -= dx

        if evidence < zStart:
            evidence = zStart
            
        # store new value of evidence at current timestep
        trace[nsteps] = evidence
            
        if evidence >= a:
            # calculate RT (in milliseconds)
            rt = tr + (nsteps * dt)
            # set choice to 1.0 (upper bound)
            choice = 1.0

            # terminate simulation, return rt & choice
            return rt, choice

    # return -1.0 for rt and choice so we can filter out
    # trials where evidence never crossed 0 or a
    return -1.0, -1.0

### Background code for running DDM and generating dataframes

In [ ]:
def gen_ddm_storage_objects(parameters, ntrials=200, deadline=1.5):
    dt = parameters[-1]
    ntime = np.int(np.floor(deadline / dt))
    data = np.zeros((ntrials, 2))
    rProb = random_sample((ntrials, ntime))
    traces = np.zeros_like(rProb)
    return data, rProb, traces

def clean_output(data, traces, deadline=1.2, stimulus=None):
    df = pd.DataFrame(data, columns=['rt', 'choice'])
    df.insert(0, 'trial', np.arange(1, 1+df.shape[0]))
    df = df[(df.rt>0)&(df.rt<deadline)]
    traces = traces[df.index.values, :]
    df = df.reset_index(drop=True)
    return df, traces

@jit((float64[:], float64[:,:], float64[:,:], float64[:,:]), nopython=True)
def _sim_ddm_trials_(parameters, data, rProb, traces):
    ntrials = data.shape[0]
    for t in range(ntrials):
        data[t, :] = sim_ddm(parameters, rProb[t], traces[t])

def sim_ddm_trials(parameters, ntrials=500, deadline=1.5, decay=False):
    data, rProb, traces = gen_ddm_storage_objects(parameters, ntrials, deadline)
    _sim_ddm_trials_(parameters, data, rProb, traces)
    df, traces = clean_output(data, traces, deadline=deadline)
    return df, traces

### Initialize our parameters
We can begin with a regular DDM (with one boundary) and tweak with the parameters or potentially add more parameters to get this best fit to the behavioral data that we have

In [ ]:
a = .09 # boundary height
v = .25 # strong drift-rate
tr = .15 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .1 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 2 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 100 # number of trials to simulate



parameters = np.array([a, tr, v, z, si, dx, dt])

In [ ]:
df, traces = sim_ddm_trials(parameters, ntrials, deadline)
df.head(10)

In [ ]:
accuracy = df.choice.mean()
corRT = df[df.choice==1].rt.mean()
y,x = df.shape
count = 0 
print(y)


while count <y:

    df["rt"][count] =  (df["rt"][count])
    
    count+=1


print("RT (cor) = {:.0f} ms".format(corRT/dt))
print("Accuracy = {:.0f}%".format(accuracy*100))

### Visualization of single barrier DDM

In [ ]:
ax = vis.plot_this_sims(df, parameters,traces = traces, plot_v=True)

### Create a single decision barrier DDM with attention decay

In [ ]:
@jit(nb.typeof((1.0, 1.0))(float64[:], float64[:], float64[:]), nopython=True)
def sim_decay_ddm(parameters, rProb, trace):
    # extract parameters
    a, tr, v, z, si, dx, dt, decay, decay_acceleration = parameters

    # convert drift-rate into a probability,
    # & scale by sigma (si) and timestep (dt)
    # if v > 0, then 0.5 < vProb < 1.0
    # if v < 0, then 0.0 < vProb < 0.5
    vProb = .5 * (1 + (v * np.sqrt(dt))/si)

    # define starting point with respect to boundary height
    zStart = z * a

    #initialize evidence variable at zStart
    evidence = zStart
    trace[0] = evidence

    # define deadline (max time allowed for accumulation)
    deadline = trace.size

    for nsteps in range(1, deadline):
        # sample a random probability (r) and compare w/ vProb
        if rProb[nsteps] < vProb:
            # if r < vProb, step evidence up (towards a)
            evidence += dx
        else:
            # if r > vProb, step evidence down (towards 0)
            evidence -= dx

        # always add in decay parameter
        evidence -= (decay + decay_acceleration * Trial)
            
        if evidence < zStart:
            evidence = zStart
            
        # store new value of evidence at current timestep
        trace[nsteps] = evidence
            
        if evidence >= a:
            # calculate RT (in milliseconds)
            rt = tr + (nsteps * dt)
            # set choice to 1.0 (upper bound)
            choice = 1.0

            # terminate simulation, return rt & choice
            return rt, choice

    # return -1.0 for rt and choice so we can filter out
    # trials where evidence never crossed 0 or a
    return -1.0, -1.0

In [ ]:
df.head(10)

### Background code for running decay DDM and generating dataframes

In [ ]:
@jit((float64[:], float64[:,:], float64[:,:], float64[:,:]), nopython=True)
def _sim_decay_ddm_trials_(parameters, data, rProb, traces):
    ntrials = data.shape[0]
    for t in range(ntrials):
        Trial = t
        data[t, :] = sim_decay_ddm(parameters, rProb[t], traces[t])

def sim_decay_ddm_trials(parameters, ntrials=500, deadline=1.5, decay=False):
    data, rProb, traces = gen_ddm_storage_objects(parameters, ntrials, deadline)
    _sim_decay_ddm_trials_(parameters, data, rProb, traces)
    df, traces = clean_output(data, traces, deadline=deadline)
    return df, traces

In [ ]:
a = .3 # boundary height
v = .40 # strong drift-rate
tr = 0 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .1 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 5 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 1000 # number of trials to simulate
decay = 0.000005
decay_acceleration = 0.00001


parameters_decay = np.array([a, tr, v, z, si, dx, dt, decay, decay_acceleration])


In [ ]:
decayDf, traces = sim_decay_ddm_trials(parameters_decay, ntrials, deadline)
meanRT = decayDf.rt.mean()

stdDevRT = decayDf.rt.std()

print("RT (average) = {:.0f} ms".format(meanRT/dt))
print('stdDev = {:.0f} ms'.format(stdDevRT/dt))

In [ ]:
bx = vis.plot_this_sims(decayDf, parameters,traces = traces, plot_v=False)

## Our Approach to Model Accuracy Estimation 
Now that we've created a single boundary DDM capable of producing simulated Reaction Time distribution, we want to begin manipulating its parameters to fit it to our experimental data. In order to do that in a principled way, however, we need a way of measuring if a particular set of parameters produces a better fit on the experimental data than another set of parameters. For this we will use a Chi Square 'goodness of fit' test. 

#### TODO: $$Formula For Chi Square Goes Here$$ 

The goal for our model is to minimize the value of the Chi Square, as it represents the likelihood that the distribution produced by the model is the same distribution from the experimental data.

### Chi Square to Calculate Goodnes of Fit
At a conceptual level, we are comparing how closely the observed (experimental) data matches the expected (simulation) data. We calculate the probability of any differences between the datasets being present, if they are indeed sampled from the same distribution. 

#### TODO: Visualization of Observed vs Expected distribution, can use lecture slides



+ **Note** here, that because we are using a single output of the model simulation, a model that inherently has a high variance between Simualtion instances can have significantly different $\chi^2 $ value depending on which instance we use calculate this statistic. As such, averaging the distributions between simulations of such a model increases reliability. 

### 'Bucketing' the data
For us to be able to calculate the $\chi^2$ fit we need to choose RT intervals within which to 'bucket' data points. Intuitively, the more fine-grained the buckets, the snugger the fit. But there is a catch. For this calculation to have statistical validity we must have at least 5 data points within each bucket. While this is not a problem for simulation data, it is a limitation for our experimental data since we only have so many trials. 

This is a key constraint that has downstream effects on what we choose to model and what comparisons we draw. Particularly, individual subject modeling now becomes a challnge, as we only have 47 data points for the entire experiment (3 for the pre-test period, 41 for the experimental period, and 3 for the post-test period). This is mostly due to flaws in the experimental design, and of course, the researchers have no clue that the data was going to be modeled in this way. 

#### TODO: Present our RT intervals and the rationale/process for arriving at them 



### The algorithm for calculating the goodness of fit
Special thanks to Wikipedia, Khan Academy and Tim and Kyle for the surprinsingly informative vizualization of the $\chi^2$ estimation on the DDM lecture slides. 

1. Define the buckets for your RT intervals. 
2. Collect your data points into the buckets. Do this for the experimental data and the simulated data (using the same intervals for both data sets).  
3. Count the Data points inside the intervals to make sure there is more than 5 in each bucket (Large Counts condition)
4. For each interval calculate $(observed counts - expected counts)/expected counts$, and then sum them all to calculate the Chi Square of your model. 
$$ \chi^s = SumFormulaHere $$
5. Get the degrees of freedom of this Chi Square distribution by subtracting one from your number of buckets. 
6. Compare your Chi Square against the critical Chi Square values for different values of $\alpha$. They represent the likelihood that your simulated and observed data come from the same distribution. 
    * There is a $\alpha$ percent chance that if the two datasets come from the same distribution, their differences would be what they are or more extreme



## Setting up goodness of fit calculation

### Importing and Formatting experimental data

In [ ]:
import pandas as pd
dataPath = "data/RTData.csv"
data = pd.read_csv(dataPath, header = 0)
data = data.dropna()

dataWhole = data[data.isFreq == 1]
dataisV = dataWhole[dataWhole.isVar==1]
dataisNV = dataWhole[dataWhole.isVar==0]
dataisInit = data[data.timePeriod == 'Initial']
dataisEnd = data[data.timePeriod == 'End']

def changeD(rData): 
    rRT = "clickDelay"
    rt = "rt"
    trial = []
    newRT = []
    newCh = []
    dShape,garb = rData.shape
    trial = [i for i in range(dShape)]
    newRT = list(rData[rRT])

    newCh = [1 for i in range(dShape)]
    newDf = {"trial":trial,"rt":newRT,"choice":newCh}
    return pd.DataFrame(data = newDf)
dataAll,dataVar,dataNVar,dataInit,dataEnd = changeD(dataWhole),changeD(dataisV),changeD(dataisNV),changeD(dataisInit),changeD(dataisEnd)

#add identifiers 
dataAll['id'] = 'All'
dataVar['id'] = 'Var'
dataNVar['id'] = 'NVar'

### Defining the Buckets
Our first thought to define our 'buckets' was by setting them to be the width of one standard deviation, with two 'catch all' buckets for the distribution tails. After some testing, these proved to be a bit too wide so we set them to half of a standard deviation. 

We experimented by using the standard error $\sigma/\sqrt{N}$, but it proved too small. 

After gathering the RT counts within the buckets we realized we had to collapse the initial buckets into one because the contained too few counts. As such they would violate the large counts assumption of the Chi Square model and ruin our **statistical validity**. This is due to the experimental data having a rightward skew. 

#### Our initial buckets
Experimental buckets:
+ All data buckets =  [0, 0, 0, 16, 108, 70, 69, 44, 27, 20, 15, 16, 31]
+ Variable condition buckets =  [0, 0, 0, 22, 54, 33, 31, 30, 14, 9, 12, 8, 20]
+ Sable condition buckets =  [0, 0, 0, 2, 47, 43, 30, 18, 10, 8, 3, 6, 16]

We need at least 5 data points in each bucket. 

In [ ]:
import math

def stdDev(mean,L):
    Sum = 0
    for i in L: 
        Sum+=math.pow((i-mean),2)
    return math.sqrt((Sum/len(L)))


def bucketer(mean,stdD,n):
    conMax = 3 #confidence rating 
    n = math.sqrt(n)
    bucket = []
    low = -1*conMax
    high = conMax+1 
    for i in [j for j in range(low,high)]:
        term = mean + i*(stdD/n)
        bucket.append(term)
    return bucket

def sampleBasedBucketer(mean, stdD):
    distance = 6 # 2 std devs, since it will be halved below
    bounds = []
    for i in range(-distance, distance):
        term = mean + i * (stdD/3)
        bounds.append(term)    
    return bounds


def getSampleBounds(panDf):
    mean = panDf.rt.mean()
    stdDev = panDf.rt.std()
    print("sample Mean = {:.3f} s".format(mean))
    print("sample Std Dev = {:.3f} s".format(stdDev))
    return sampleBasedBucketer(mean, stdDev)
           
def getStats(panDf): 
    y,x = panDf.shape
    Sum = 0
    count =0
    new = []
    while count<y:
        ind = panDf["rt"][count]
        Sum+= ind
        new+=[ind]
        count+=1
    mean = Sum/count
    std = stdDev(mean,new)
    n = len(new)
    buck = bucketer(mean,std,n)
    return mean,std,buck

testM,testD,testB = getStats(decayDf)

sampleBounds = getSampleBounds(decayDf)

#print('std Error buckets')
#for bound in testB:
#    print(bound)

#print('sample std Dev buckets')
#for bound in sampleBounds:
#    print(bound)
    
print("simulated Mean = {:.3f} s".format(testM))
print("simulated Std Dev = {:.3f} s".format(testD))

#### Count the buckets into an histogram-like array.

In [ ]:
def isInBounds(lower, upper, value):
    return lower <= value and value <= upper

def getBuckets(bounds, dataframe):
    buckets = [0] * (len(bounds) + 1) 
    bounds.sort()
    boundsList = [] # [(lower1, upper1), (lower2, upper2) ...]
    for i,bound in enumerate(bounds):
        if i == 0:
            boundsList.append((0, bound))
        else:
            boundsList.append((bounds[i-1], bound))
        #add right tail end bucket
        if i == len(bounds) - 1:
            boundsList.append((bound, 100)) #100 is a catchAll 
            
    #print(dataframe.shape)
    for i,(lower, upper) in enumerate(boundsList):
        for rt in dataframe.rt.values:
            if isInBounds(lower, upper, rt):
                buckets[i] += 1
    return buckets

obsvMean,obsvDev,obsvBuck = getStats(dataAll)

sampleObsvBuck = getSampleBounds(dataAll)

simBucketCount = getBuckets(testB, decayDf)
obsvBucketCount = getBuckets(obsvBuck, dataAll)
obsvSampleBucketCount = getBuckets(sampleObsvBuck, dataAll)
simSampleBucketCount = getBuckets(sampleBounds, decayDf)



#print("simulated buckets: \n", simBucketCount)
print("simulated buckets sample based: \n", simSampleBucketCount)


print("\n All data experimental buckets: \n", obsvBucketCount)
print("All experimental  buckets sample based: \n", obsvSampleBucketCount)

#using sample based from now on

print("\nExperimental buckets:")
allDataBuckets = getBuckets(getSampleBounds(dataAll), dataAll)
varDataBuckets = getBuckets(getSampleBounds(dataVar), dataVar)
staDataBuckets = getBuckets(getSampleBounds(dataNVar), dataNVar)
print("All data buckets = ", allDataBuckets)
print("Variable condition buckets = ", getBuckets(getSampleBounds(dataVar), dataVar))
print("Sable condition buckets = ", getBuckets(getSampleBounds(dataNVar), dataNVar))

#### Calculate Chi Square
while making sure to not violete the Large Counts rule

In [ ]:
## calculate Chi squares
import scipy
from scipy import stats

def collapseUpToFive(L):
    firstCount = L[0]
    i = 1
    while firstCount <= 5:
        firstCount += L[i]
        i += 1
    return i - 1

#print(collapseUpToFive([0, 0, 0, 22, 54, 33, 31, 30, 14, 9, 12, 8, 20]))
#print(collapseUpToFive([0, 0, 0, 2, 47, 43, 30, 18, 10, 8, 3, 6, 16]))


def getChiSquare(observedDf, expectedDf):
    #get ObservedBounds
    bounds = getSampleBounds(observedDf)
    # get buckets for both using observedBounds
    observedBuckets = getBuckets(bounds, observedDf)
    truncator = collapseUpToFive(observedBuckets)
    adjustedBounds = bounds[truncator:]
    observedBuckets = getBuckets(adjustedBounds, observedDf)
    expectedBuckets = getBuckets(adjustedBounds, expectedDf)
    
    assert(len(observedBuckets) == len(expectedBuckets))
    
    
    
    observed_values = scipy.array(observedBuckets)
    expected_values = scipy.array(expectedBuckets)
    chiSquare, pValue = scipy.stats.chisquare(observed_values, f_exp=expected_values)
    print("observed buckets =", observedBuckets)
    print("expected buckets = ", expectedBuckets)
    print("chi Square = {:3f}".format(chiSquare))
    print("pvalue = {:3f}".format(pValue))
    return chiSquare, pValue 

#test chi square:
getChiSquare(dataVar, dataVar) 




### Vizualize the data to build intuition

In [ ]:
#plotting the raw Data 
allDataAx = sns.distplot(dataAll.rt.values).set_title('All Data')

#sample Mean = 3.026 s
#sample Std Dev = 2.304 s
#adj bounds =  [0.7220, 1.4900, 2.2581, 3.02624, 3.7943, 4.5623, 5.330, 6.09854, 6.8666]

In [ ]:
# Non Variable Statistics
#sample Mean = 3.200 s
#sample Std Dev = 2.540 s
#adj bounds (sec) = , 1.507, 2.354, 3.200, 4.047, 4.893, 5.740, 6.586, 7.433

staDataAx = sns.distplot(dataNVar.rt.values, color='r').set_title('Non Var')

In [ ]:
# Variable Statistics
#sample Mean = 2.890 s
#sample Std Dev = 2.096 s
#adj bounds (sec) =  0.793, 1.492, 2.191, 2.890, 3.588, 4.287, 4.986, 5.685, 6.384

varDataAx = sns.distplot(dataVar.rt.values, color='g').set_title('Var')

In [ ]:
#plotting the raw Data for var and sta
varDataAx = sns.distplot(dataVar.rt.values, color='g', label='Var')
staDataAx = sns.distplot(dataNVar.rt.values, color='r', label='Non Var')
legend, title = plt.legend(), plt.title('variable vs Stable')

## Parameter Estimation

#### Simple Single-Bound DDM - Variable

In [ ]:
a = .5 # boundary height
v = .0235 # strong drift-rate
tr = .15 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .3 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 10 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 233 # number of trials to simulate



parameters = np.array([a, tr, v, z, si, dx, dt])

simVarDf, traces = sim_ddm_trials(parameters, ntrials, deadline)
cs, p = getChiSquare(dataVar, simVarDf)

Sum = 0

for test in range(100):
    meanRT = simVarDf.rt.mean()
    stdDevRT = simVarDf.rt.std()

    print("RT (average) = {:.0f} ms".format(meanRT/dt))
    print('stdDev = {:.0f} ms'.format(stdDevRT/dt))
    simVarDf, traces = sim_ddm_trials(parameters, ntrials, deadline)
    cs,p = getChiSquare(dataVar, simVarDf)
    print()
    Sum += p
print("Mean p-value:", (Sum/100))

varx = vis.plot_this_sims(df, parameters,traces = traces, plot_v=True)

#### Decay Single-Bound DDM - Variable

In [ ]:
a = .5 # boundary height
v = .0235 # strong drift-rate
tr = .15 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .3 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 10 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 233 # number of trials to simulate
decay = 0.000005
decay_acceleration = 0.00001


parameters_decay = np.array([a, tr, v, z, si, dx, dt, decay, decay_acceleration])

simDecayVarDf, traces = sim_decay_ddm_trials(parameters_decay, ntrials, deadline)
cs, p = getChiSquare(dataVar, simDecayVarDf)

Sum = 0

for test in range(100):
    meanRT = simDecayVarDf.rt.mean()
    stdDevRT = simDecayVarDf.rt.std()

    print("RT (average) = {:.0f} ms".format(meanRT/dt))
    print('stdDev = {:.0f} ms'.format(stdDevRT/dt))
    simDecayVarDf, traces = sim_decay_ddm_trials(parameters_decay, ntrials, deadline)
    cs,p = getChiSquare(dataVar, simDecayVarDf)
    print()
    Sum += p
print("Mean p-value:", (Sum/100))

varx = vis.plot_this_sims(df, parameters,traces = traces, plot_v=True)

#### Simple Single-Bound DDM - Non Variable

In [ ]:
a = .5 # boundary height
v = .0235 # strong drift-rate
tr = .15 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .3 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 10 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 233 # number of trials to simulate



parameters = np.array([a, tr, v, z, si, dx, dt])

simNVarDf, traces = sim_ddm_trials(parameters, ntrials, deadline)
cs, p = getChiSquare(dataNVar, simNVarDf)

Sum = 0

for test in range(100):
    meanRT = simNVarDf.rt.mean()
    stdDevRT = simNVarDf.rt.std()

    print("RT (average) = {:.0f} ms".format(meanRT/dt))
    print('stdDev = {:.0f} ms'.format(stdDevRT/dt))
    simNVarDf, traces = sim_ddm_trials(parameters, ntrials, deadline)
    cs,p = getChiSquare(dataVar, simNVarDf)
    print()
    Sum += p
print("Mean p-value:", (Sum/100))

# varx = vis.plot_this_sims(df, parameters,traces = traces, plot_v=True)

#### Decay Single-Bound DDM - Non Variable

In [ ]:
a = .5 # boundary height
v = .0235 # strong drift-rate
tr = .15 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .3 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 10 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 233 # number of trials to simulate
decay = 0.000005
decay_acceleration = 0.00001


parameters_decay = np.array([a, tr, v, z, si, dx, dt, decay, decay_acceleration])

simDecayNVarDf, traces = sim_decay_ddm_trials(parameters_decay, ntrials, deadline)
cs, p = getChiSquare(dataVar, simDecayNVarDf)

Sum = 0

for test in range(100):
    meanRT = simDecayNVarDf.rt.mean()
    stdDevRT = simDecayNVarDf.rt.std()

    print("RT (average) = {:.0f} ms".format(meanRT/dt))
    print('stdDev = {:.0f} ms'.format(stdDevRT/dt))
    simDecayNVarDf, traces = sim_decay_ddm_trials(parameters_decay, ntrials, deadline)
    cs,p = getChiSquare(dataNVar, simDecayNVarDf)
    print()
    Sum += p
print("Mean p-value:", (Sum/100))

# varx = vis.plot_this_sims(df, parameters,traces = traces, plot_v=True)

# TODO

+ Estimate parameters for models (show before and after to show improvement in chi squares)
    + write function that automatically calculates the chi squared against a defined experimental data set (all, var, Nvar)
    + MVP (DDM for Var vs Nvar)
+ Beyond 1: QQ plots (prolly not lol)
+ Results section
+ Conclusion 
+ Write Background and Problem sections

# Results

# Conclusion

## Future Work 

### Plots for initial vs End data 

notice how the end distribution is shifted more towards the right

In [ ]:
dataisInit = data[data.timePeriod == 'Initial']
dataisEnd = data[data.timePeriod == 'End']
dataInit,dataEnd = changeD(dataisInit),changeD(dataisEnd)

In [ ]:
initialColor = '#FCC800'
initDataAx = sns.distplot(dataInit.rt.values, color='orange', label='Initial')

In [ ]:
endColor = '#00479D'
endDataAx = sns.distplot(dataEnd.rt.values, color='b', label='End')

In [ ]:
initDataAx = sns.distplot(dataInit.rt.values, color='orange', label='Initial')
endDataAx = sns.distplot(dataEnd.rt.values, color='b', label='End')
legend, title = plt.legend(), plt.title('Initial Period vs End')